# Symbolic expression for the amplitude model

$$
\begin{eqnarray}
A^{12} &=& \frac{\sum a_m Y_2^m (\Omega_1)}{s-m^2_{a_2}+im_{a_2} \Gamma_{a_2}} \\
A^{23} &=& \frac{\sum b_m Y_1^m (\Omega_2)}{s-m^2_{\Delta}+im_{\Delta} \Gamma_{\Delta}}  \\
\
A^{31} &=& \frac{c_0}{s-m^2_{N^*}+im_{N^*} \Gamma_{N^*}}
\end{eqnarray}
$$ (BW_SH_label)

In [ ]:
import graphviz
import sympy as sp
from sympy import (
    re,
    im,
    I,
    E,
    symbols,
    Sum,
    IndexedBase,
    Function,
    Ynm,
    Symbol,
    simplify,
)
from IPython.display import display, Latex

In [ ]:
s, m_a2, Gamma_a2 = symbols(r"s m_{a_2} \Gamma_{a_2}")
l, m, theta1, phi1 = symbols("l m theta_1 phi_1")
Ynm(l, m, theta, phi)
a_m = IndexedBase("a_m")
Y2_m = Function("Y_2^m")("Omega_1")
m = symbols("m", cls=sp.Idx)
Omega1 = symbols("Omega_1")

A12 = sp.Sum(a_m * Ynm(2, m, theta1, phi1), (m, -2, 2)) / (
    s - m_a2**2 + I * m_a2 * Gamma_a2
)
A12

In [ ]:
m_Delta, Gamma_Delta = symbols("m_Delta Gamma_Delta")
b_m = IndexedBase("b_m")
Y1_m = Function("Y_1^m")("Omega_2")
theta2, phi2 = symbols("theta_2 phi_2")
A23 = Sum(b_m * Ynm(1, m, theta2, phi2), (m, -1, 1)) / (
    s - m_Delta**2 + I * m_Delta * Gamma_Delta
)
A23

In [ ]:
c0, m_Nstar, Gamma_Nstar = symbols(r"c_0 m_{N^*} \Gamma_{N^*}")
# r'\Gamma_{N^*}'
A31 = c0 / (s - m_Nstar**2 + I * m_Nstar * Gamma_Nstar)
A31